In [1]:
#import libraries
import rundic as rd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from numpy import loadtxt
from numpy import NaN

ModuleNotFoundError: No module named 'fast_interp'

In [8]:
#-------------------------------------------------------------------------------------
##load settings and image sets
settings = rd.LoadSettings()
image_set_0, image_set_1 = rd.LoadStereoImages(settings)


print('\nsorted image set\n')
print(len(image_set_0))
print('\n', len(image_set_1))

ROI_0 = (1243, 661, 312, 1178)



Initial settings:

{'MeasurementType': 'StereoPlanar', 'Discretisation': 'Local', 'StereoImagesFolder0': '../../TENSILE_IMAGES/L', 'StereoImagesFolder1': '../../TENSILE_IMAGES/R', 'InitializeViewMatch': 'Projective', 'RefineViewMatch': 'No', 'CameraNo': 0, 'CalibrationParametersFolder': 'Sample5Calibration', 'SubsetSize': 31, 'DeformationModel': 'Affine', 'StepSize': 3, 'SubsetShape': 'Square', 'ImportMeasurementPointsFromExternal': 'Yes', 'GaussianFilterStdDev': 0.5, 'GaussianFilterSize': 5, 'CorrelationReferenceStrategy': 'Incremental', 'DatumImage': 0, 'TargetImage': 96, 'Increment': 96, 'InterpolateReferenceImage': 'No', 'GNInitialization': 'KeypointDetection'}

Stereo images loaded

image_set_0:
['L0000.png', 'L0001.png', 'L0003.png', 'L0005.png', 'L0007.png', 'L0009.png', 'L0011.png', 'L0013.png', 'L0015.png', 'L0017.png', 'L0019.png', 'L0021.png', 'L0023.png', 'L0025.png', 'L0027.png', 'L0029.png', 'L0031.png', 'L0033.png', 'L0035.png', 'L0037.png', 'L0039.png', 'L0041.png', 'L

In [9]:
#perform planar dic on the left and right image sets
coefficients_0, coefficients_1, settings, data = rd.StereoDICPlanar(settings, image_set_0, image_set_1, ROI_0)
#load the calibration parameters from directory
settings, data = rd.ReadCalibrationParamters(settings, data)
#perform stereo triangulation on the reference and deformed coordinates (in the camera CS)
settings, data = rd.DepthFromTwoViewsCameraCS(settings, data)
#two points on an axis that aligns with the specimen's short axis
#used to define local coordinate system
xo_local_axis = np.array([[1223, 1542],
                            [1488, 1493]]).astype('int')
#reference, deformed measuremenet point coordinates and displacements in the locally define CS
Xo_w, Xd_w, U_w = rd.BestPlaneFit(settings, data, image_set_0, image_set_1, xo_local_axis)
#compute Green-Lagrange strain tensor from the 3D displacements
Ex, Ey, Exy = rd.VirtualStrainGauge(settings, Xo_w[0, :], Xo_w[1, :], U_w[0, :], U_w[1, :])


no. of image pairs: 1
Image pair 0 processed

left camera image series correlation complete

no. of image pairs: 1
Image pair 0 processed

right camera image series correlation complete


In [ ]:
#pre-process data for plots

F = cv.imread('L_images/L0000.png', 0)
N_rows = F[600:1900, 1200:1600].shape[0]
N_cols = F[600:1900, 1200:1600].shape[1]

xo = settings['MeasurementPoints_L0'][0, :]
yo = settings['MeasurementPoints_L0'][0, :]
N_subsets = xo.shape[0]

#structure the results to gird format
XY = np.zeros([N_subsets, 2])
for i in range(0, N_subsets):
    XY[i, :] = (yo[i]-600, xo[i]-1200)
grid_x, grid_y = np.mgrid[0:N_cols:1, 0:N_rows:1]


invalid_x = loadtxt('invalid_x.csv', delimiter = ",").astype('int')
invalid_y = loadtxt('invalid_y.csv', delimiter = ",").astype('int')

invalid_x = invalid_x +5
invalid_y = invalid_y +5

invalid_x = invalid_x - 1200
invalid_y = invalid_y - 600

#linecut x coordinate for plots
line_indices = np.where(xo==1287)  
y_line = yo[line_indices]

In [ ]:
#y-displacement
ey_new_grid = griddata(XY, Ey, (grid_y, grid_x), method='cubic')
ey_new_grid = ey_new_grid.T
ey_new_grid[invalid_y, invalid_x] = NaN
#
plt.imshow(F[600:1900, 1200:1600], cmap = 'gray')
plt.imshow(ey_new_grid, cmap = 'jet', aspect = '1')
#plt.colorbar(location = 'bottom', shrink = 0.4, ticks = [-0.155, -0.085, -0.01]).set_label('mm')
plt.axis('off')
cbar = plt.colorbar(location = 'bottom', shrink = 0.15, pad = 0.05)
cbar.set_ticks([np.round(np.min(Ey), decimals = 3), np.round(np.max(Ey), decimals = 3)])

plt.tight_layout()
#plt.savefig('ey_contour.pdf', bbox_inches='tight',pad_inches = 0.01)
plt.show()

In [ ]:
#linecut
fig, axs = plt.subplots()
fig.tight_layout()
axs.plot(y_line, Ey[line_indices])
axs.set_ylabel('$\epsilon_{yy}$ (-)')
axs.set_xlabel('\nY (pixels)')
axs.legend('eyy strain (Green-Lagrange)')
axs.grid()
plt.show()